In [ ]:
import pandas as pd
import datetime
import plotly
import plotly.express as px
import plotly.graph_objects as go

#chartTemplate = 'plotly_dark'
chartTemplate = 'plotly_white'

In [ ]:
# Datos de Cuba de https://ourworldindata.org/covid-cases

#casos = pd.read_csv("owid-covid-data.csv")
casos = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv")

cuba0 = casos[casos["location"] == "Cuba"][["date","new_cases"]]
cuba = cuba0.reset_index().rename(columns={'date': 'Date','new_cases':'N'}).sort_values(by='Date', ascending=True)
cuba

,index,Date,N
0,26377,2020-03-12,3.0
1,26378,2020-03-13,1.0
2,26379,2020-03-14,0.0
3,26380,2020-03-15,0.0
4,26381,2020-03-16,0.0
...,...,...,...
555,26932,2021-09-18,8517.0
556,26933,2021-09-19,8434.0
557,26934,2021-09-20,8544.0
558,26935,2021-09-21,8289.0


In [ ]:
cases = pd.read_csv("covid19-casos.csv")

cases_cfg = cases[cases["provincia"] == "Cienfuegos"]

#cuba = cases['fecha_confirmacion'].value_counts().reset_index().rename(columns={'index': 'Date','fecha_confirmacion':'N'}).sort_values(by='Date', ascending=True)
cfg = cases_cfg['fecha_confirmacion'].value_counts().reset_index().rename(columns={'index': 'Date','fecha_confirmacion':'N'}).sort_values(by='Date', ascending=True)

# calculate case velocity
#cuba["vel"] = cuba.loc[:, cuba.columns != 'Date'].diff()
cuba["vel"] = cuba.loc[:, 'N'].diff()
cfg["vel"] = cfg.loc[:, 'N'].diff()

# calculate case acceleration
cuba["acc"] = cuba[["Date", "vel"]].loc[:, 'vel'].diff()
cfg["acc"] = cfg[["Date", "vel"]].loc[:, 'vel'].diff()

# calculate means - 14 días - 2 semanas
# usando rolling
avg = 14
cfg["accelMeans14"] = cfg[["Date", "vel"]].loc[:, 'vel'].rolling(window = avg).mean()
cfg["accelMeans7"] = cfg[["Date", "vel"]].loc[:, 'vel'].rolling(window = 7).mean()
cfg["accelMeans3"] = cfg[["Date", "vel"]].loc[:, 'vel'].rolling(window = 3).mean()
cuba["accelMeans14"] = cuba[["Date", "vel"]].loc[:, 'vel'].rolling(window = avg).mean()
cuba["accelMeans7"] = cuba[["Date", "vel"]].loc[:, "vel"].rolling(window = 7).mean()
cuba["accelMeans3"] = cuba[["Date", "vel"]].loc[:, "vel"].rolling(window = 3).mean()

# con EWM
# cfg["accelMeans5"] = cfg[["Date", "vel"]].loc[:, cfg[["Date", "vel"]].columns != 'Datetime'].ewm(span=5).mean()

cuba

/shared-libs/python3.9/py-core/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning:

Columns (1,7) have mixed types.Specify dtype option on import or set low_memory=False.



,index,Date,N,vel,acc,accelMeans14,accelMeans7,accelMeans3
0,26377,2020-03-12,3.0,NaN,NaN,NaN,NaN,NaN
1,26378,2020-03-13,1.0,-2.0,NaN,NaN,NaN,NaN
2,26379,2020-03-14,0.0,-1.0,1.0,NaN,NaN,NaN
3,26380,2020-03-15,0.0,0.0,1.0,NaN,NaN,-1.000000
4,26381,2020-03-16,0.0,0.0,0.0,NaN,NaN,-0.333333
...,...,...,...,...,...,...,...,...
555,26932,2021-09-18,8517.0,226.0,-437.0,47.357143,68.857143,360.000000
556,26933,2021-09-19,8434.0,-83.0,-309.0,-56.214286,57.142857,268.666667
557,26934,2021-09-20,8544.0,110.0,193.0,93.857143,28.857143,84.333333
558,26935,2021-09-21,8289.0,-255.0,-365.0,37.000000,110.428571,-76.000000


In [ ]:
fig = px.line(cfg, x="Date", y="N", template=chartTemplate)
fig.update_layout({"title": 'Casos - Cienfuegos',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [ ]:
fig = px.line(cfg, x="Date", y="vel", template=chartTemplate)

fig.update_layout({"title": 'Velocidad de casos - Cienfuegos',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"}})
                   #"showlegend": False})
fig.show()

In [ ]:
cfg.head(5)

,Date,N,vel,acc,accelMeans14,accelMeans7
220,2020/03/24,1,NaN,NaN,NaN,NaN
188,2020/03/27,3,2.0,NaN,NaN,NaN
219,2020/03/28,1,-2.0,-4.0,NaN,NaN
185,2020/03/29,3,2.0,4.0,NaN,0.666667
197,2020/04/01,3,0.0,-2.0,NaN,0.000000


In [ ]:
#import numpy as np
# here you define the color
#cfg["color"] = np.where(cfg["accelMeans14"] >= 0, 'green', "red")
## add a color column to the df, apply along row
#cfg['color'] = cfg.apply(lambda x: 'red' if x['accelMeans14'] >= 0 else 'green', axis=1)
pos = cfg[cfg['accelMeans7'] >= 0]
neg = cfg[cfg['accelMeans7'] < 0]

fig = go.Figure()

fig = px.line(cfg, x="Date", y="accelMeans14", template=chartTemplate)

#fig.add_trace(go.Bar(x=cfg['Date'], y=cfg['accelMeans14'], name='Promedio de 14 días', marker = dict(color = 'red')) 

fig.add_trace(go.Bar(x=pos['Date'], y=pos['accelMeans7'], name='Promedio de 14 días', marker = dict(color = 'red'), opacity=0.5))
fig.add_trace(go.Bar(x=neg['Date'], y=neg['accelMeans7'], name='Promedio de 14 días', marker = dict(color = 'green'), opacity=0.5))#fig.update_layout({"title": 'Aceleración de casos - promedio 14 días - Cienfuegos', "xaxis": {"title":"Días"},"yaxis": {"title":"Casos"}, "showlegend": False})
fig.show()

#  ##--

# CUBA

## Análisis de Cuba

#  ##--

In [ ]:
fig = px.line(cuba, x="Date", y="N", template=chartTemplate)

fig.update_layout({"title": 'Casos - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [ ]:
fig = px.line(cuba, x="Date", y="vel", template=chartTemplate)

fig.update_layout({"title": 'Velocidad - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [ ]:
fig = px.line(cuba, x="Date", y="acc", template=chartTemplate)

fig.update_layout({"title": 'Aceleración de casos - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})

fig.show()

In [ ]:
pos = cuba[cuba['accelMeans7'] >= 0]
neg = cuba[cuba['accelMeans7'] < 0]

fig = px.line(cuba, x="Date", y="accelMeans14", template=chartTemplate)

fig.update_layout({"title": 'Aceleración de casos - promedio 14 días - Cuba',
                   "xaxis": {"title":"Días"},
                   "yaxis": {"title":"Casos"},
                   "showlegend": False})
fig.add_trace(go.Bar(x=pos['Date'], y=pos['accelMeans7'], name='Promedio de 14 días', opacity=0.5))
fig.add_trace(go.Bar(x=neg['Date'], y=neg['accelMeans7'], name='Promedio de 14 días', opacity=0.5))

fig.show()

In [ ]:
#bars = 'accelMeans7'
bars = 'accelMeans3'
#bars = 'acc'
#bars = 'vel'

pos = cuba[cuba[bars] >= 0]
neg = cuba[cuba[bars] < 0]

fig = go.Figure()

### Traces
# Line
fig.add_trace(go.Scatter(
    x = cuba['Date'],
    y = cuba['accelMeans14'],
    name = 'Aceleración 14d',
    line = dict(
        color = ('blue'),
        width = 1)
))
# Bars
fig.add_trace(go.Bar(
    x=pos['Date'], 
    y=pos[bars], 
    name='Acc positiva 3d', # Positive velocity
    marker = dict(color = 'red'), 
    opacity=0.7
))
fig.add_trace(go.Bar(
    x=neg['Date'], 
    y=neg[bars], 
    name='Acc negativa 3d', # Negative velocity
    marker = dict(color = 'green'), 
    opacity=0.7
))

### Layout
layout = {'title': 'Aceleración de casos - promedio móvil 14 días (línea) - 3 días (barras) - Cuba',
            #"width" : 800,
            #"height" : 600
         }
fig.update_layout(layout)
fig.show()

In [ ]:
#bars = 'accelMeans7'
bars = 'accelMeans3'
#bars = 'acc'
#bars = 'vel'

pos = cuba[cuba[bars] >= 0]
neg = cuba[cuba[bars] < 0]

#fig = go.Figure()

from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

### Traces
# Line
fig.add_trace(go.Scatter(
    x = cuba['Date'],
    y = cuba['accelMeans14'],
    name = 'Aceleración 14d',
    line = dict(
        color = ('blue'), #lambda x:  'red' if x < 0 else 'green'
        width = 1),
), secondary_y=False)
fig.add_trace(go.Bar(
    x=pos['Date'], 
    y=pos[bars], 
    name='Acc positiva 3d', # Positive velocity
    marker = dict(color = 'red'), 
    opacity=0.5
), secondary_y=True)
fig.add_trace(go.Bar(
    x=neg['Date'], 
    y=neg[bars], 
    name='Acc negativa 3d', # Negative velocity
    marker = dict(color = 'green'), 
    opacity=0.7, yaxis = 'y2'
), secondary_y=True)

### Layout
layout = {
            'title': 'Aceleración de casos - promedio móvil 14 días (línea) - 3 días (barras) - Cuba',
            #"width" : 800,
            #"height" : 600,
            'yaxis': {
                    'zerolinecolor': '#969696',
                    'zerolinewidth': 2,
                    'rangemode': 'tozero'
            },
            'yaxis2': {
                    'overlaying': 'y',
                    'side': 'right',
                    'rangemode': 'tozero'
             },
            'yaxis3': {
                    'overlaying': 'y',
                    'side': 'right',
                    'rangemode': 'tozero'

             }
}
fig.update_yaxes(secondary_y=False,
                 tickfont_color="MediumSlateBlue", title="Promedio 14d - Línea")
fig.update_yaxes(secondary_y=True,
                 tickfont_color="DarkOrange", title="Promedio 3d - Barras")
fig.update_layout(layout)
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=331fe212-3056-4b5e-ab8f-59cd76bdab29' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>